# Introduction to Lyncs

This is an introductory notebook to Lyncs.

It contains some examples describing the main features of the API.

Lyncs is a Python API for lattice field theory simulations with the main focus on lattice QCD.

The lattice field thaory is represented by the `lyncs.Lattice` object. It contains all the information needed for the simulation.

In [1]:
import lyncs as ly
lattice = ly.Lattice(dims=4, dofs="QCD")
lattice

Lattice(dims = {'t': 1, 'x': 1, 'y': 1, 'z': 1},
        dofs = {'spin': 4, 'color': 3},
        dtype = dtype('complex128'),
        properties = {'time': ['t'], 'space': ['x', 'y', 'z'], 'gauge_dofs': ['color']})

In [2]:
lattice.t = 8
lattice['x'] = 4
lattice.space = 4
lattice

Lattice(dims = {'t': 8, 'x': 4, 'y': 4, 'z': 4},
        dofs = {'spin': 4, 'color': 3},
        dtype = dtype('complex128'),
        properties = {'time': ['t'], 'space': ['x', 'y', 'z'], 'gauge_dofs': ['color']})

In [6]:
field = ly.Field(lattice=lattice, field_type="vector")
field.shape

[('t', 8), ('x', 4), ('y', 4), ('z', 4), ('spin', 4), ('color', 3)]

In [7]:
field

Field(array = Delayed('Field-bf3521f2-3ef7-438f-8225-101b135dc916'),
      lattice = Lattice(dims = {'t': 8, 'x': 4, 'y': 4, 'z': 4},
                        dofs = {'spin': 4, 'color': 3},
                        dtype = dtype('complex128'),
                        properties = {'time': ['t'], 'space': ['x', 'y', 'z'], 'gauge_dofs': ['color']}),
      field_type = 'vector',
      extra_options = {})

In [5]:
ly.load("conf.1000")

Field(array = Delayed('read_data-da8e05b8-867c-44ef-b47f-0da45e84f155'),
      lattice = Lattice(dims = {'t': 4, 'x': 4, 'y': 4, 'z': 4},
                        dofs = {'spin': 4, 'color': 3},
                        dtype = dtype('complex128'),
                        properties = {'time': ['t'], 'space': ['x', 'y', 'z'], 'gauge_dofs': ['color']}),
      field_type = 'gauge_links',
      extra_options = {})

In [2]:
field = ly.Field(lattice=lattice, field_type="vector")
field

Field(array = Delayed('Field-1c03dc0b-4568-49f4-8f45-18f6731fdeca'),
      lattice = Lattice(dims = {'t': 1, 'x': 1, 'y': 1, 'z': 1},
                        dofs = {'spin': 4, 'color': 3},
                        dtype = dtype('complex64'),
                        properties = {'gauge_dofs': ['color']}),
      field_type = 'vector',
      extra_options = {})